In [1]:
# all import statements needed for the project, for example:
import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import pandas
import requests 
import sqlalchemy as db
from pandas import read_parquet
from pyarrow.parquet import ParquetDataset
from bs4 import BeautifulSoup
import re
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
from numba import cuda,jit
from datetime import datetime
import datetime
import glob
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session
import matplotlib.pyplot as plt
import sqlite3
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import datetime

In [2]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
redius = 6378.137
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"
TAXI_DIR = 'taxi_files'
QUERY_DIRECTORY = "queries"
WEATHER_DIR = "weather_files"

zones_df =gpd.read_file(r".\taxi_zones\taxi_zones.shp")
zones_df.to_csv("taxi_zones.csv")
zones_df = zones_df.to_crs(4326)
zones_df["lat"] = zones_df.centroid.x
zones_df["lon"] = zones_df.centroid.y
zones_data = []

zones_pickupID = zones_df.drop(
        ["OBJECTID","Shape_Leng",
         "Shape_Area","zone",
         "borough","geometry"],
        axis="columns").rename(
        columns={
        "LocationID":"PULocationID",
        "lon":"pickup_latitude",
        "lat":"pickup_longitude"
        })
zones_dropoffID = zones_df.drop([
                "OBJECTID","Shape_Leng",
                "Shape_Area","zone",
                "borough","geometry"],
                axis="columns").rename(
                columns={
                "LocationID":"DOLocationID",
                "lon":"dropoff_latitude",
                "lat":"dropoff_longitude"
                })

In [11]:

# Calculate the distance through coordinates
def calculate_distance(from_coord, to_coord):
    lat1, lon1 = from_coord
    lat2, lon2 = to_coord

    diff_lat = math.radians(lat2 - lat1)
    diff_lon = math.radians(lon2 - lon1)
    
    cal_dis = (math.sin(diff_lat / 2) * math.sin(diff_lat / 2) +
             math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
             math.sin(diff_lon / 2) * math.sin(diff_lon / 2))
    cal_dis = 2 * math.atan2(math.sqrt(cal_dis), math.sqrt(1 - cal_dis))
    distance = redius * cal_dis

    return distance
    

# Add the coordinate columns to the dataframe
def add_distance_column(dataframe):
    dataframe["trip_distance"] = dataframe.apply(
                                lambda row: calculate_distance((row.pickup_longitude, row.pickup_latitude), 
                                                               (row.dropoff_longitude, row.dropoff_latitude)), 
                                                                axis=1)
    return dataframe

In [14]:
### Part 1: Data Preprocessing

In [15]:
# Get the download links of yellow taxi from 2009-01 to 2015-06
def find_taxi_csv_urls():
    response = requests.get(TAXI_URL)
    soup = BeautifulSoup(response.text, "html.parser")
    url_tags = soup.find_all("a", title="Yellow Taxi Trip Records")
    all_csv_urls = []
    pattern = "2009|2010|2011|2012|2013|2014|2015-0[1-6]"

    for url_tag in url_tags:
        if re.findall(pattern, url_tag["href"]):
            all_csv_urls.append(url_tag["href"])

    return all_csv_urls

In [ ]:
def load_data_from_filename(filename):
    data = pd.read_parquet(filename)
    return data

In [ ]:
def get_and_clean_month_taxi_data(url:str):
    file_name = url.split('/')[-1]
    if not os.path.exists(f'{TAXI_DIR}/{file_name}'):
        with open(f'{TAXI_DIR}/{file_name}', 'wb') as f:
            res = requests.get(url)
            f.write(res.content)

    dataframe = load_data_from_filename(f'{TAXI_DIR}/{file_name}')
    # Delete the useless columns
    # Attention: year 2009 and 2010 have unexpected columns' names.
    if "2010-" in file_name:
        dataframe = dataframe[
                            (dataframe["passenger_count"]> 0)
                            & (dataframe["total_amount"] > 0)
                                ]
        
        dataframe.drop([
                        "vendor_id", "dropoff_datetime", "rate_code", "store_and_fwd_flag",
                        "payment_type", "fare_amount", "surcharge",
                        "mta_tax","tolls_amount","passenger_count", 
                        "total_amount"
                        ], axis=1, inplace=True)
    elif "2009-" in file_name:
        dataframe = dataframe[
                            (dataframe["Passenger_Count"]> 0)
                            & (dataframe["Total_Amt"] > 0)
                                ]
        
        dataframe.drop([
                        "vendor_name", "Trip_Dropoff_DateTime", "Rate_Code",
                        "store_and_forward", "Payment_Type", "Fare_Amt", 
                        "surcharge", "mta_tax","Tolls_Amt",
                        "Passenger_Count",  'Total_Amt'
                        ], axis=1, inplace=True)
        
    else:
        dataframe = dataframe[
                            (dataframe["passenger_count"]> 0)
                            & (dataframe["total_amount"] > 0)
                                ]
        
        dataframe.drop([
                        "RatecodeID", "tolls_amount", "payment_type", 
                        "store_and_fwd_flag","mta_tax", "improvement_surcharge", 
                        "fare_amount", "extra","congestion_surcharge",
                        "airport_fee","VendorID", "tpep_dropoff_datetime", 
                        "passenger_count", "total_amount"
                        ], axis=1, inplace=True)
                        
        # Unify the columns' names of year 2009 and 2012
    if "2010-" in file_name or "2009-" in file_name:
         dataframe.columns = [
                            "pickup_datetime", "trip_distance", "pickup_longitude", 
                             "pickup_latitude", "dropoff_longitude", "dropoff_latitude", 
                             "tip_amount"
                            ]
            
    else:
        # Change the IDs to the pick-up and drop-off coordinates, adding columns
        df = pd.merge(dataframe, zones_pickupID, on = "PULocationID")
        dataframe = pd.merge(df, zones_dropoffID, on = "DOLocationID")
        dataframe.drop(["PULocationID", "DOLocationID"], axis=1, inplace=True)
        dataframe.columns = [
                            "pickup_datetime", "trip_distance", "tip_amount",
                            "pickup_latitude", "dropoff_longitude", "dropoff_latitude",
                            "tip_amount"
                            ]
    
    # Remove the invalide datas
    dataframe = dataframe[
                            (dataframe["pickup_longitude"] >= -74.242330)
                            & (dataframe["pickup_latitude"] <= 40.908524)
                            & (dataframe["pickup_longitude"] <= -73.717047)
                            & (dataframe["pickup_latitude"] >= 40.560445)
                            & (dataframe["dropoff_longitude"] >= -74.24233)
                            & (dataframe["dropoff_latitude"] <= 40.908524)
                            & (dataframe["dropoff_longitude"] <= -73.717047)
                            & (dataframe["dropoff_latitude"] >= 40.560445)
                            ]
    
    # Randomly choose a sample from every month.
    # Accumulated sample size is approximately to the uber data amount
    dataframe = dataframe.sample(n=2600)
    
    return dataframe                

In [ ]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    df_list = []
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        dataframe = dataframe[dataframe["trip_distance"]> 0]
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        _df = dataframe
        _df.to_sql("taxi_trips", con=engine, index=False, if_exists="append")
        df_list.append(_df)
    
    
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    taxi_data.to_csv(f'finally_taxi_data.csv', index=False, header=taxi_data.columns, encoding="utf-8")

In [ ]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    df = df.drop(columns = ["key", "Unnamed: 0"])
    df = df[
            (df["pickup_longitude"] >= -74.242330)
            & (df["pickup_latitude"] <= 40.908524)
            & (df["pickup_longitude"] <= -73.717047)
            & (df["pickup_latitude"] >= 40.560445)
            & (df["dropoff_longitude"] >= -74.24233)
            & (df["dropoff_latitude"] <= 40.908524)
            & (df["dropoff_longitude"] <= -73.717047)
            & (df["dropoff_latitude"] >= 40.560445)
            & (df["passenger_count"] > 0)
            & (df["fare_amount"] > 0 )
            ]

    df.drop(["passenger_count","fare_amount"], axis=1, inplace=True)
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"]).apply(lambda t: t.replace(tzinfo=None))
    
    return df    

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    uber_dataframe= uber_dataframe[uber_dataframe["trip_distance"] > 0]
    return uber_dataframe

In [ ]:
def add_uber_data_to_db():
    uber_data = get_uber_data()
    uber_data.to_csv(f'finally_uber_data.csv', index=False, header=uber_data.columns, encoding='utf-8')
    
    for i in range(40):
        uber_df = uber_data[i * 5000:(i + 1)*5000]
        uber_df.to_sql('uber_trips', con=engine, index=False, if_exists='append')
        


In [ ]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file)
    df2 = df.loc[:, ["DATE", "HourlyWindSpeed",'HourlyPrecipitation']]
    df2["DATE HOUR"] = df2["DATE"].apply(lambda x:x[:-6])
    df2.drop(columns = ["DATE"], axis=1, inplace=True)
    df2["HourlyPrecipitation"].replace("T", 0.001)
    
    df2["HourlyWindSpeed"] = df2["HourlyWindSpeed"].apply(pd.to_numeric, errors = "coerce")
    df2["HourlyPrecipitation"] = df2["HourlyPrecipitation"].apply(pd.to_numeric, errors = "coerce")
    
    df2["HourlyWindSpeed"].fillna(0, inplace=True)
    df2["HourlyPrecipitation"].fillna(0, inplace=True)
    
    df3= df2.groupby(["DATE HOUR"], as_index=False)["HourlyWindSpeed"].mean()
    
    df4= df2.groupby(["DATE HOUR"], as_index=False)["HourlyPrecipitation"].sum()
    
    df4= pd.merge(df3, df4, on="DATE HOUR", how="inner")
    
    df4["DATE HOUR"] = list(map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%dT%H").strftime("%Y-%m-%d %H:"), df4["DATE HOUR"]))
    
    return df4

In [ ]:
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(csv_file)
    df2 = df.loc[:, ["DATE", "HourlyWindSpeed",'HourlyPrecipitation']]
    df2["DATE DAYLY"] = df2["DATE"].apply(lambda x:x[:-9])
    df2.drop(columns=["DATE"],axis= 1,inplace=True)
    df2["HourlyPrecipitation"].replace('T', 0.001)
    
    df2["HourlyWindSpeed"] = df2["HourlyWindSpeed"].apply(pd.to_numeric, errors = "coerce")
    df2["HourlyPrecipitation"] = df2["HourlyPrecipitation"].apply(pd.to_numeric, errors = "coerce")
    
    
    df2["HourlyWindSpeed"].fillna(0, inplace=True)
    df2["HourlyPrecipitation"].fillna(0, inplace=True)
        
    df3= df2.groupby(["DATE DAYLY"], as_index=False)["HourlyWindSpeed"].mean()
    
    df4= df2.groupby(["DATE DAYLY"], as_index=False)["HourlyPrecipitation"].sum()
    
    df4= pd.merge(df3, df4, on="DATE DAYLY", how='inner')
    df4["DatelyWindSpeed"] = df4["HourlyWindSpeed"]
    df4["DatelyPrecipitation"] = df4["HourlyPrecipitation"]
    
    
    df4.drop(columns=["HourlyPrecipitation","HourlyWindSpeed"],axis= 1,inplace=True)
    
    return df4

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []

    weather_csv_files = [
                       "2009_weather.csv","2010_weather.csv","2011_weather.csv",
                       "2012_weather.csv","2013_weather.csv","2014_weather.csv",
                       "2015_weather.csv"
                        ]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    daily_data.drop(daily_data.loc[daily_data['DATE DAYLY'] > "2015-06-31"].index, inplace=True)
    hourly_data.drop(hourly_data.loc[hourly_data['DATE HOUR'] > "2015-06-31"].index, inplace=True)
    
    hourly_data.columns = ["datetime", "wind_speed", "precipitation"]
    daily_data.columns = ["datetime", "wind_speed", "precipitation"]
    return hourly_data, daily_data

In [ ]:
def add_weather_data_to_db():
    hourly_data, daily_data = load_and_clean_weather_data()

    daily_data.to_sql("daily_weather", con=engine, index=False, if_exists="append")
    daily_data.to_csv(f'finally_daily_weather_data.csv', index=False, header=daily_data.columns, encoding="utf-8")

    # Import the hourly weather data to the table in 13 batches and
    # every time has amount of around 5000
    hourly_data.to_csv(f'finally_hourly_weather_data.csv', index=False, header=hourly_data.columns, encoding="utf-8")
    for i in range(13):
        hourly_df = hourly_data[i * 5000:(i + 1)*5000]
        hourly_df.to_sql("hourly_weather", con=engine, index=False, if_exists='append')
        